Cloning the git hub file python classifier which is given part of the Project

In [ ]:
!git clone https://github.com/physionetchallenges/python-classifier-2022.git pythonClassifier2022
!git clone https://github.com/physionetchallenges/evaluation-2022.git

fatal: destination path 'pythonClassifier2022' already exists and is not an empty directory.
fatal: destination path 'evaluation-2022' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing the libraries.

In [ ]:
import numpy as np
import librosa as lb
from tqdm import tqdm
import torch
import os
import pandas as pd
from pythonClassifier2022.helper_code import *
import IPython.display as ipd
import librosa.display
from scipy import signal
from os import listdir
from IPython.display import Audio
import seaborn as sns
from tqdm import tqdm
import warnings
# ignore all warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,ParameterGrid
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer,confusion_matrix, f1_score, balanced_accuracy_score, precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay

In [ ]:
!cp '/content/drive/MyDrive/surrey/aihealth/Project_AI_Health/the-circor-digiscope-phonocardiogram-dataset-1.0.3.zip' '/content/data.zip'
!unzip '/content/data.zip'

Archive:  /content/data.zip
replace the-circor-digiscope-phonocardiogram-dataset-1.0.3/LICENSE.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Analysis of the data present in the text file 

In [ ]:
#Reading one of the text file in the training dataset.

df_patient=open('the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/84877.txt','r')

print(df_patient.read())

In [ ]:
#Total how many patient details present in the dataset.

path_train = '/content/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/'
patient_files = find_patient_files(path_train)
len(patient_files)

In [ ]:
# we are checking what all the content present in the text file.
current_patient_data = load_patient_data(patient_files[1])
current_patient_data

In [ ]:
#it will be searching the Murmur whether it is present in the text file or not if it is there it will return with the value .

def extract_label(data, search_str):
    murmur = None
    for l in data.split('\n'):
        if l.startswith('#Murmur:'):
            try:
                murmur = l.split(': ')[1]
            except:
                pass
    if murmur is None:
        raise ValueError('No murmur available')
    return murmur

In [ ]:
#we are extracting the value associated with each label in the text file.
#we are going through each and every line.
def get_specific_info(data, search_str):
    for line in data.split('\n'):
        if search_str in line:
            label = line.split(': ')[1]
            return label
    raise ValueError('Error: search string not found')


In [ ]:
# we are extracting features from the text file we are arranging them in proper data structure format we are labeling them.
# with the help of python classifier we are extracting values from the text file.

info='''#Age,
#Sex,
#Height,
#Weight,
#Pregnancy status,
#Murmur locations,
#Most audible location,
#Systolic murmur timing,
#Systolic murmur shape,
#Systolic murmur grading,
#Systolic murmur pitch,
#Systolic murmur quality,
#Diastolic murmur timing,
#Diastolic murmur shape,
#Diastolic murmur grading,
#Diastolic murmur pitch,
#Diastolic murmur quality,
#Campaign'''.split(',\n')

def get_patient_info(data):
    id = get_patient_id(data)
    num_loc = get_num_locations(data)
    locations = get_locations(data)
    frec = get_frequency(data)
    #age = get_age(data)
    #sex = get_sex(data)
    #height = get_height(data)
    #weight = get_weight(data)
    #pregnancy = get_pregnancy_status(data)
    info_v=[]
    murmur = get_murmur(data)
    for i in info:
        info_v.append(get_specific_info(data,i))
    return np.hstack((id,info_v,frec,num_loc,'+'.join(locations),murmur))

In [ ]:
#we are extracting the values assigned to the variables.

get_patient_info(current_patient_data)

In [ ]:
#total there are 23 columns.

len(get_patient_info(current_patient_data))

In [ ]:
#extracting values from all the text file .

database = np.zeros((len(patient_files), 23), dtype='<U32')
for i, patient in enumerate(patient_files):
    patient_info = get_patient_info(load_patient_data(patient))
    patient_info_int = np.array(patient_info)
    database[i] = patient_info_int

df=pd.DataFrame(database,columns=['id']+info+['frequency','num_loc','locations','Murmur'])
df.head()


Knowing type of data present in the columns

In [ ]:
#we are just analysing the output label .
#size of the each label.

data = df['Murmur']
plt.hist(data, bins=10)
plt.xlabel('Murmur')
plt.title('Histogram of Murmur')
plt.show()

In [ ]:
#we are seeing how many null values present in the age.
#similary we have null values in height ,weight pregnancy.
count_age = df['#Age'].value_counts()
count_nan = df['#Age'].isnull().sum()

print(count_age)
print(count_nan)

In [ ]:
count_sex = df['#Sex'].value_counts()
count_nan = df['#Sex'].isnull().sum()

print(count_sex)
print(count_nan)

In [ ]:
count_pregnancy = df['#Pregnancy status'].value_counts()
count_nan = df['#Pregnancy status'].isnull().sum()

print(count_pregnancy)
print(count_nan)

Signal Analysis

In [ ]:
scale_AV='the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/84877_AV.wav'

In [ ]:
scale_MV='the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/84877_MV.wav'

In [ ]:
scale_PV='the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/84877_PV.wav'

In [ ]:
scale_TV='the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/84877_TV.wav'

In [ ]:
ipd.Audio(scale_AV)

In [ ]:
ipd.Audio(scale_MV)

In [ ]:
ipd.Audio(scale_PV)

In [ ]:
ipd.Audio(scale_TV)

In [ ]:
#sr=none  It means original sample
#y is raw data of the audio file
#sr is sample rate
y,sr=lb.load('the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data/84877_AV.wav')
print(f'y: {y[:20]}')
print(f'y shape : {y.shape}')
print(f'sr : {sr}')


In [ ]:
pd.Series(y[0:100]).plot(figsize=(10,5),lw=5,title='raw audio sample')
plt.show()

In [ ]:
#short time fourier transform
#which frequency is sounding in the audio file by using the fourier transform
#amplitude to decibels
d=lb.stft(y)
df_db=lb.amplitude_to_db(np.abs(d),ref=np.max)
df_db.shape

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))

img=lb.display.specshow(df_db,x_axis='time',y_axis='log',ax=ax)


In [ ]:
#mfccs will capture the frequency content of the audio signal.
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

In [ ]:
#displaying the audio file 
plt.figure(figsize=(10,5))
lb.display.specshow(mfccs,x_axis='time',sr=sr)
plt.show()


Extracting the Features from the Audio File.

In [ ]:
#Rms is useful for detecting changes in the intensity of the sound.
#audio will be in analog form we will convert into the digital form and we are extracting the features from the audio .
#Signal is the original audio signal
#Window is how many seconds we are considering 
#Rate is the sampling size of the audio.
#we are extracting the features we are rearranging the shape of the features in proper way.
#we are joining all the features.
#from each of the features we are considering their min ,max ,mean,median and standard deviation.it will caluclate across all time frames.
#For "Mfcc" we are only considering the default value which is 13.

import librosa

def audio_features(signal, rate, window):
    n_fft = int(rate * window)
    hop_length = int(n_fft / 2)
    
    #extraacting the features
    stft = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(stft)
    mfccs = librosa.feature.mfcc(y=signal, sr=rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    zcr = librosa.feature.zero_crossing_rate(signal, frame_length=n_fft, hop_length=hop_length)[0]
    rms = librosa.feature.rms(S=S_mag, frame_length=n_fft, hop_length=hop_length)[0]
    sc = librosa.feature.spectral_centroid(y=signal, sr=rate, S=S_mag, n_fft=n_fft, hop_length=hop_length)[0]

    # Concatenate and aggregate features
    features = np.concatenate([mfccs, zcr.reshape(1, -1), rms.reshape(1, -1), sc.reshape(1, -1)])
    min_features = np.min(features, axis=1)
    max_features = np.max(features, axis=1)
    median_features = np.median(features, axis=1)
    mean_features = np.mean(features, axis=1)
    std_features = np.std(features, axis=1)
    all_features = np.concatenate([min_features, max_features, median_features, mean_features, std_features])
    
    return all_features


One hot encoding the categorical columns and also presenting the Audio features and demographic features

In [ ]:
#In this we are one hot encoding the all the categorical values.and also splitting the most audible location recording location and most loud location in the individual columns . 
#we are separting all the audio features extracted into the signal features all the demographic features into the demo features.


def get_specific_info(data, search_str):

    label = None
    for l in data.split('\n'):
        if l.startswith(search_str):
            try:
                label = l.split(': ')[1]
            except:
                pass
    if label is None:
        raise ValueError('error when searching for more info')
    return label



def get_patient_info(data,info):
    id = get_patient_id(data)
    locations = get_locations(data)
    info_v=[]
    for i in info:
        info_v.append(get_specific_info(data,i))
    return id,locations,info_v 



def get_features(data_folder,data_path,resamp,w,info):

    data = load_patient_data(data_path)
    id,locations,info_v  = get_patient_info(data,info)
    


    age_group=info_v[0]
    if compare_strings(age_group, 'Neonate'):
        age = 0
    elif compare_strings(age_group, 'Infant'):
        age = 1
    elif compare_strings(age_group, 'Child'):
        age = 2
    elif compare_strings(age_group, 'Adolescent'):
        age = 3
    elif compare_strings(age_group, 'Young Adult'):
        age = 4
    else:
        age = float('nan')


    #get sex

    if compare_strings(info_v[1], 'Female'):
        sex = 0
    elif compare_strings(info_v[1], 'Male'):
        sex = 1
    else:
        sex = float('nan')
    
    #get heigth and weight
    height = info_v[2]
    weight = info_v[3]


    #get pregnancy status
    if compare_strings(info_v[4], 'False'):
        preg = 0
    elif compare_strings(info_v[4], 'True'):
        preg = 1
    else:
        preg = float('nan')

    recording_locations = ['AV', 'MV', 'PV', 'TV', 'Phc']

    if len(info_v)>5:
        murmur_locs = info_v[5].split('+')
        murmur_locs_cod = np.zeros(5,dtype=int)
        murmur_most_au = np.zeros(5,dtype=int)
        for _,loc in enumerate(murmur_locs):
            if loc in recording_locations:
                murmur_locs_cod[recording_locations.index(loc)]=+1

        if info_v[6] in recording_locations:
            murmur_most_au[recording_locations.index(info_v[6])]=+1
        demo_features = np.hstack((age,sex,height,weight,preg,murmur_locs_cod,murmur_most_au))
    else:
        demo_features = np.hstack((age,sex,height,weight,preg))

    
    signal_features = list()
    num_locations = get_num_locations(data)
    recording_information = data.split('\n')[1:num_locations+1]

    for c,i in enumerate(locations):
        entries = recording_information[c].split(' ')
        locs = np.zeros(5,dtype=int)
        recording_file = entries[2]
        if i in recording_locations:
            locs[recording_locations.index(i)]=+1
        else:
            print(_,i)
        x,rate=lb.load(data_folder+recording_file,sr=resamp)
        x=x/np.max(np.abs(x))
        signal_features.append( np.hstack( (id,locs,audio_features(x,rate,wind)) ) )

    return demo_features,signal_features

We are using different audio length and sampling structure . and also extracting all the features from the all the audio files

In [ ]:
#we are also tested with the 0.3 o.4 and 0.5 and also we also tried with the different sampling size of 500 , 1000 ,1500 we thought of considering of 0.2 with sampling size of 1000
#it will giving the best accuracy score and f1 score.

wind=0.2
resamp=1000
all_features = []
info='''#Age,
#Sex,
#Height,
#Weight,
#Pregnancy status,
#Murmur locations,
#Most audible location'''.split(',\n')

patient_files = find_patient_files(path_train)
demo_data=[]
signal_data=[]
for i in tqdm(patient_files):
    current=get_features(path_train,i,resamp,wind,info)
    demo_data.append(current[0])
    signal_data.extend(current[1])

100%|██████████| 942/942 [00:35<00:00, 26.76it/s]


In [ ]:
#labeling them and also arranging them in a proper way all the audio features which we are extracted from the analog audio file.
label='''id,AV,MV,PV,TV,Phc,
mfcc1 min, mfcc2 min, mfcc3 min, mfcc4 min, mfcc5 min, mfcc6 min, mfcc7 min, mfcc8 min, mfcc9 min, mfcc10 min, mfcc11 min, mfcc12 min, mfcc13 min, zcr min, rms min, sc min,
mfcc1 max, mfcc2 max, mfcc3 max, mfcc4 max, mfcc5 max, mfcc6 max, mfcc7 max, mfcc8 max, mfcc9 max, mfcc10 max, mfcc11 max, mfcc12 max, mfcc13 max, zcr max, rms max, sc max,
mfcc1 med, mfcc2 med, mfcc3 med, mfcc4 med, mfcc5 med, mfcc6 med, mfcc7 med, mfcc8 med, mfcc9 med, mfcc10 med, mfcc11 med, mfcc12 med, mfcc13 med, zcr med, rms med, sc med,
mfcc1 mean, mfcc2 mean, mfcc3 mean, mfcc4 mean, mfcc5 mean, mfcc6 mean, mfcc7 mean, mfcc8 mean, mfcc9 mean, mfcc10 mean, mfcc11 mean, mfcc12 mean, mfcc13 mean, zcr mean, rms mean, sc mean,
mfcc1 std, mfcc2 std, mfcc3 std, mfcc4 std, mfcc5 std, mfcc6 std, mfcc7 std, mfcc8 std, mfcc9 std, mfcc10 std, mfcc11 std, mfcc12 std, mfcc13 std, zcr std, rms std, sc std'''
label = label.replace('\n',' ')
char=np.array(signal_data,dtype=np.float64)
signal_features=pd.DataFrame(char,columns=label.split(','))

It will representing all the audio features

In [ ]:
signal_features.head()

,id,AV,MV,PV,TV,Phc,mfcc1 min,mfcc2 min,mfcc3 min,mfcc4 min,...,mfcc7 std,mfcc8 std,mfcc9 std,mfcc10 std,mfcc11 std,mfcc12 std,mfcc13 std,zcr std,rms std,sc std
0,2530.0,1.0,0.0,0.0,0.0,0.0,-492.886963,-74.591644,-56.831894,-37.606979,...,12.292173,9.925873,9.209851,9.216714,8.690586,7.139463,7.038485,0.116124,0.020789,51.887637
1,2530.0,0.0,0.0,1.0,0.0,0.0,-474.274170,44.373314,-37.568436,-19.823332,...,10.347335,11.242581,9.494909,8.644834,6.965057,7.800419,7.301554,0.035099,0.017048,26.762607
2,2530.0,0.0,0.0,0.0,1.0,0.0,-505.165955,-2.775817,-54.885117,-18.885195,...,10.336310,8.422961,8.798788,7.295115,7.513894,6.191885,6.380693,0.040040,0.013724,36.500468
3,2530.0,0.0,1.0,0.0,0.0,0.0,-465.794159,-15.000306,-75.130394,-42.193970,...,10.352138,8.331804,9.191442,6.949651,8.119811,6.947865,6.764353,0.058066,0.014341,27.025486
4,9979.0,1.0,0.0,0.0,0.0,0.0,-419.479218,-0.709317,-47.496399,-13.292309,...,8.531240,8.868077,9.253156,7.356440,8.199459,8.376468,7.963311,0.040838,0.026153,26.413326


In [ ]:
signal_features.shape

(3163, 86)

In [ ]:
#labelig them and also arranging them in proper way
label1='''age,sex,height,weight,preg,murmur_locs_AV,murmur_locs_MV,murmur_locs_PV,murmur_locs_TV,murmur_locs_Phc,murmur_most_AV,murmur_most_MV,murmur_most_PV,murmur_most_TV,murmur_most_Phc'''
char1=np.array(demo_data,dtype=np.float64)
demo_features=pd.DataFrame(char1,columns=label1.split(','))

It will be representing the demographic features

In [ ]:
demo_features.head()

,age,sex,height,weight,preg,murmur_locs_AV,murmur_locs_MV,murmur_locs_PV,murmur_locs_TV,murmur_locs_Phc,murmur_most_AV,murmur_most_MV,murmur_most_PV,murmur_most_TV,murmur_most_Phc
0,2.0,0.0,98.0,15.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,103.0,13.1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,1.0,115.0,19.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,1.0,98.0,15.9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.0,1.0,87.0,11.2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
demo_features.shape

(942, 15)

In [ ]:
#adding the patient id in future it will helpful in merging
signal_features['Patient ID']=signal_features['id'].apply(lambda x: int(x))

In [ ]:
signal_features.head()

,id,AV,MV,PV,TV,Phc,mfcc1 min,mfcc2 min,mfcc3 min,mfcc4 min,...,mfcc8 std,mfcc9 std,mfcc10 std,mfcc11 std,mfcc12 std,mfcc13 std,zcr std,rms std,sc std,Patient ID
0,2530.0,1.0,0.0,0.0,0.0,0.0,-492.886963,-74.591644,-56.831894,-37.606979,...,9.925873,9.209851,9.216714,8.690586,7.139463,7.038485,0.116124,0.020789,51.887637,2530
1,2530.0,0.0,0.0,1.0,0.0,0.0,-474.274170,44.373314,-37.568436,-19.823332,...,11.242581,9.494909,8.644834,6.965057,7.800419,7.301554,0.035099,0.017048,26.762607,2530
2,2530.0,0.0,0.0,0.0,1.0,0.0,-505.165955,-2.775817,-54.885117,-18.885195,...,8.422961,8.798788,7.295115,7.513894,6.191885,6.380693,0.040040,0.013724,36.500468,2530
3,2530.0,0.0,1.0,0.0,0.0,0.0,-465.794159,-15.000306,-75.130394,-42.193970,...,8.331804,9.191442,6.949651,8.119811,6.947865,6.764353,0.058066,0.014341,27.025486,2530
4,9979.0,1.0,0.0,0.0,0.0,0.0,-419.479218,-0.709317,-47.496399,-13.292309,...,8.868077,9.253156,7.356440,8.199459,8.376468,7.963311,0.040838,0.026153,26.413326,9979


We are reading the csv file.Provided along with the training dataset

In [ ]:
#we will analysis the csv file
data=pd.read_csv(r'/content/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv')
data.head()

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN


In [ ]:
#checking the columns name.
data.columns

Index(['Patient ID', 'Recording locations:', 'Age', 'Sex', 'Height', 'Weight',
       'Pregnancy status', 'Murmur', 'Murmur locations',
       'Most audible location', 'Systolic murmur timing',
       'Systolic murmur shape', 'Systolic murmur grading',
       'Systolic murmur pitch', 'Systolic murmur quality',
       'Diastolic murmur timing', 'Diastolic murmur shape',
       'Diastolic murmur grading', 'Diastolic murmur pitch',
       'Diastolic murmur quality', 'Outcome', 'Campaign', 'Additional ID'],
      dtype='object')

We are combining the demo features from the text file with Csv file

In [ ]:
df_data = data[['Patient ID','Age', 'Sex', 'Height', 'Weight','Pregnancy status','Murmur','Murmur locations', 'Outcome']]
df_demo = demo_features[['murmur_most_AV','murmur_most_MV','murmur_most_PV','murmur_most_TV','murmur_most_Phc']]

In [ ]:
#combining the demo features which we are extracted from the text file and demographic data from csv file.
new_data = pd.concat([df_data,df_demo], axis=1)
new_data

,Patient ID,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Outcome,murmur_most_AV,murmur_most_MV,murmur_most_PV,murmur_most_TV,murmur_most_Phc
0,2530,Child,Female,98.0,15.9,False,Absent,NaN,Abnormal,0.0,0.0,0.0,0.0,0.0
1,9979,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,Abnormal,0.0,0.0,0.0,1.0,0.0
2,9983,Child,Male,115.0,19.1,False,Unknown,NaN,Abnormal,0.0,0.0,0.0,0.0,0.0
3,13918,Child,Male,98.0,15.9,False,Present,TV,Abnormal,0.0,0.0,0.0,1.0,0.0
4,14241,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,Abnormal,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,Child,Male,105.0,16.6,False,Absent,NaN,Normal,0.0,0.0,0.0,0.0,0.0
938,85341,Child,Male,92.0,15.2,False,Absent,NaN,Normal,0.0,0.0,0.0,0.0,0.0
939,85343,Child,Female,97.0,13.5,False,Present,MV+TV,Abnormal,0.0,0.0,0.0,1.0,0.0
940,85345,Child,Female,132.0,38.1,False,Absent,NaN,Normal,0.0,0.0,0.0,0.0,0.0


In [ ]:
# we are using the dummy values for the categorical values in the dataset.
new_data = pd.get_dummies(new_data,columns=['Age', 'Sex','Pregnancy status'])
new_data

,Patient ID,Height,Weight,Murmur,Murmur locations,Outcome,murmur_most_AV,murmur_most_MV,murmur_most_PV,murmur_most_TV,murmur_most_Phc,Age_Adolescent,Age_Child,Age_Infant,Age_Neonate,Sex_Female,Sex_Male,Pregnancy status_False,Pregnancy status_True
0,2530,98.0,15.9,Absent,NaN,Abnormal,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,0,1,0
1,9979,103.0,13.1,Present,AV+MV+PV+TV,Abnormal,0.0,0.0,0.0,1.0,0.0,0,1,0,0,1,0,1,0
2,9983,115.0,19.1,Unknown,NaN,Abnormal,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1,1,0
3,13918,98.0,15.9,Present,TV,Abnormal,0.0,0.0,0.0,1.0,0.0,0,1,0,0,0,1,1,0
4,14241,87.0,11.2,Present,AV+MV+PV+TV,Abnormal,0.0,0.0,1.0,0.0,0.0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,105.0,16.6,Absent,NaN,Normal,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1,1,0
938,85341,92.0,15.2,Absent,NaN,Normal,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1,1,0
939,85343,97.0,13.5,Present,MV+TV,Abnormal,0.0,0.0,0.0,1.0,0.0,0,1,0,0,1,0,1,0
940,85345,132.0,38.1,Absent,NaN,Normal,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,0,1,0


We are replacing the missing values or Nan with the KNN imputer

In [ ]:
#we are using the knnimputer we will replacing the Nan values using knn in the height,weight,age,sex,pregnancy.
from sklearn.impute import KNNImputer
# calling the KNN class
knn_imputer = KNNImputer(n_neighbors=2)

In [ ]:
#after using the knn imputer converted values again reassigning to the datafram.
new_data[['Height','Weight','Age_Adolescent','Age_Child','Age_Infant','Age_Neonate','Sex_Female','Sex_Male','Pregnancy status_False','Pregnancy status_True']]=knn_imputer.fit_transform(new_data[['Height','Weight','Age_Adolescent','Age_Child','Age_Infant','Age_Neonate','Sex_Female','Sex_Male','Pregnancy status_False','Pregnancy status_True']])

In [ ]:
# murmur_dic = {'Absent':0,'Present':1,'Unknown':0}
murmur_dic = {'Present':0,'Unknown':1,'Absent':2}
outcome_dic = {'Abnormal': 0, 'Normal':1}
new_data['murmur_v']=new_data['Murmur'].apply(lambda x: murmur_dic[x])
new_data['outcome_v']=new_data['Outcome'].apply(lambda x: outcome_dic[x])

In [ ]:
new_data

,Patient ID,Height,Weight,Murmur,Murmur locations,Outcome,murmur_most_AV,murmur_most_MV,murmur_most_PV,murmur_most_TV,...,Age_Adolescent,Age_Child,Age_Infant,Age_Neonate,Sex_Female,Sex_Male,Pregnancy status_False,Pregnancy status_True,murmur_v,outcome_v
0,2530,98.0,15.9,Absent,NaN,Abnormal,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2,0
1,9979,103.0,13.1,Present,AV+MV+PV+TV,Abnormal,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0
2,9983,115.0,19.1,Unknown,NaN,Abnormal,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0
3,13918,98.0,15.9,Present,TV,Abnormal,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0
4,14241,87.0,11.2,Present,AV+MV+PV+TV,Abnormal,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,105.0,16.6,Absent,NaN,Normal,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2,1
938,85341,92.0,15.2,Absent,NaN,Normal,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2,1
939,85343,97.0,13.5,Present,MV+TV,Abnormal,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0
940,85345,132.0,38.1,Absent,NaN,Normal,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2,1


In [ ]:
#Aply the train_test_split on the demographic dataset
from sklearn.model_selection import train_test_split

murmurlist = list(new_data['murmur_v'])
outcomelist = list(new_data['outcome_v'])
idlist = list(new_data['Patient ID'])
demofeaturedf = new_data.drop(['Murmur', 'Outcome'], axis=1)
labellist = list(zip(murmurlist, outcomelist, idlist))

demoX_train, demoX_test, demoy_train, demoy_test = train_test_split(demofeaturedf, labellist, test_size=0.3, random_state=42)

In [ ]:
demoX_train = demoX_train.rename(columns={'Patient ID': 'id'}).astype({'id': float})
demoX_test = demoX_test.rename(columns={'Patient ID': 'id'}).astype({'id': float})

In [ ]:
trainmurmur, trainoutcome, trainidlist = zip(*demoy_train)
testmurmur, testoutcome, testidlist = zip(*demoy_test)

In [ ]:
#get all the samples from signal features having the same ID to get the signal data for train and test sets
signalX_train = signal_features[signal_features['id'].isin(trainidlist)]
signalX_test = signal_features[signal_features['id'].isin(testidlist)]

In [ ]:
#merge the signal and demo data base on the patient ID
X_train = pd.merge(demoX_train, signalX_train, on='id')
X_test = pd.merge(demoX_test, signalX_test, on='id')

In [ ]:
#count the number of cases for each murmur class
X_train['murmur_v'].value_counts()

2    1635
0     453
1     116
Name: murmur_v, dtype: int64

In [ ]:
#use specific features for the final training dataset
label='''id,AV,MV,PV,TV,Phc,
mfcc1 min, mfcc2 min, mfcc3 min, mfcc4 min, mfcc5 min, mfcc6 min, mfcc7 min, mfcc8 min, mfcc9 min, mfcc10 min, mfcc11 min, mfcc12 min, mfcc13 min, zcr min, rms min, sc min,
mfcc1 max, mfcc2 max, mfcc3 max, mfcc4 max, mfcc5 max, mfcc6 max, mfcc7 max, mfcc8 max, mfcc9 max, mfcc10 max, mfcc11 max, mfcc12 max, mfcc13 max, zcr max, rms max, sc max,
mfcc1 med, mfcc2 med, mfcc3 med, mfcc4 med, mfcc5 med, mfcc6 med, mfcc7 med, mfcc8 med, mfcc9 med, mfcc10 med, mfcc11 med, mfcc12 med, mfcc13 med, zcr med, rms med, sc med,
mfcc1 mean, mfcc2 mean, mfcc3 mean, mfcc4 mean, mfcc5 mean, mfcc6 mean, mfcc7 mean, mfcc8 mean, mfcc9 mean, mfcc10 mean, mfcc11 mean, mfcc12 mean, mfcc13 mean, zcr mean, rms mean, sc mean,
mfcc1 std, mfcc2 std, mfcc3 std, mfcc4 std, mfcc5 std, mfcc6 std, mfcc7 std, mfcc8 std, mfcc9 std, mfcc10 std, mfcc11 std, mfcc12 std, mfcc13 std, zcr std, rms std, sc std,murmur_most_AV,murmur_most_MV,murmur_most_PV,murmur_most_TV,murmur_most_Phc,Height,Weight,Age_Adolescent,Age_Child,Age_Infant,Age_Neonate,Sex_Female,Sex_Male,Pregnancy status_False,Pregnancy status_True'''
label = label.replace('\n',' ').split(',')

xfinaltrain = X_train[label].values
xfinaltest = X_test[label].values

ymurmurfinaltrain = X_train['murmur_v'].values
youtcomefinaltrain = X_train['outcome_v'].values

ymurmurfinaltest = X_test['murmur_v'].values
youtcomefinaltest = X_test['outcome_v'].values

In [ ]:
print(len(xfinaltrain))
print(len(ymurmurfinaltrain))

2204
2204


In [ ]:
#using random forest 
from sklearn.ensemble import RandomForestClassifier
rfc1 = RandomForestClassifier(n_estimators=100, random_state=42)
rfc2 = RandomForestClassifier(n_estimators=100, random_state=42)
rfc1.fit(xfinaltrain, ymurmurfinaltrain)
rfc2.fit(xfinaltrain, youtcomefinaltrain)
testmurmurproba = rfc1.predict_proba(xfinaltest)
testoutcomeproba = rfc2.predict_proba(xfinaltest)

In [ ]:
testid = np.array(list(X_test['id'])).reshape(-1, 1)

In [ ]:
# concat all ID, murmur outputs and outcome outputs and then group them base on patient ID using mean function
result = np.concatenate((testid, testmurmurproba, testoutcomeproba), axis=1)
resultdf = pd.DataFrame(result, columns=['Id', 'Present', 'Unknown', 'Absent', 'Abnormal', 'Normal'])
resultdf = resultdf.groupby(['Id']).mean()
resultdf

,Present,Unknown,Absent,Abnormal,Normal
Id,,,,,
9983.0,0.2325,0.1325,0.6350,0.5925,0.4075
14998.0,0.2350,0.1050,0.6600,0.6050,0.3950
24160.0,0.1925,0.0525,0.7550,0.5400,0.4600
31737.0,0.1875,0.0750,0.7375,0.5625,0.4375
46065.0,0.8075,0.0100,0.1825,0.7650,0.2350
...,...,...,...,...,...
85328.0,0.1100,0.0700,0.8200,0.3900,0.6100
85334.0,0.2825,0.0550,0.6625,0.4550,0.5450
85336.0,0.1300,0.2150,0.6550,0.5200,0.4800


In [ ]:
#getting the output csv file
resultdf.to_csv('randomforest.csv', index=True)

In [ ]:
#train the model on Logistic Regression
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs')
clf2 = LogisticRegression(multi_class='multinomial', solver='lbfgs')
clf1.fit(xfinaltrain, ymurmurfinaltrain)
clf2.fit(xfinaltrain, youtcomefinaltrain)

testmurmurproba = clf1.predict_proba(xfinaltest)
testoutcomeproba = clf2.predict_proba(xfinaltest)

result = np.concatenate((testid, testmurmurproba, testoutcomeproba), axis=1)
resultdf = pd.DataFrame(result, columns=['Id', 'Present', 'Unknown', 'Absent', 'Abnormal', 'Normal'])
resultdf = resultdf.groupby(['Id']).mean()
resultdf.to_csv('LogisticRegression.csv', index=True)

In [ ]:
#train the model on Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier()
clf2 = DecisionTreeClassifier()
clf1.fit(xfinaltrain, ymurmurfinaltrain)
clf2.fit(xfinaltrain, youtcomefinaltrain)

testmurmurproba = clf1.predict_proba(xfinaltest)
testoutcomeproba = clf2.predict_proba(xfinaltest)

result = np.concatenate((testid, testmurmurproba, testoutcomeproba), axis=1)
resultdf = pd.DataFrame(result, columns=['Id', 'Present', 'Unknown', 'Absent', 'Abnormal', 'Normal'])
resultdf = resultdf.groupby(['Id']).mean()
resultdf.to_csv('DecisionTree.csv', index=True)